In [133]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

train_data = pd.read_csv("./data/train.csv", encoding = 'UTF-8')
test_data = pd.read_csv("./data/test.csv", encoding='UTF-8')

train_data = train_data.drop('id', axis=1)
test_data = test_data.drop('id', axis=1)

In [134]:
# 데이터 컬럼(속성명) 이름 변경 : 데이터 컬럼을 쉽게 다루기 위해서
train_data.columns =['species','island','clutch_completion','culmen_length_mm',
                    'culmen_depth_mm','flipper_length_mm','sex','delta_15N','delta_13C','body_mass']

test_data.columns = ['species','island','clutch_completion','culmen_length_mm',
                    'culmen_depth_mm','flipper_length_mm','sex','delta_15N','delta_13C']

In [135]:
# train, test 데이터 결측치 확인
#print("train 데이터 결측치 확인 \n", train_data.isnull().sum())
#print("\n train 데이터 결측치 확인 \n", test_data.isnull().sum())

# 결측치 확인 함수 만들기
def check_missing_col(dataframe) :
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns) :
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >=1 else False
        if is_missing :
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은 : {col}입니다.')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train_data)

결측치가 있는 컬럼은 : sex입니다.
해당 컬럼에 총 3개의 결측치가 존재합니다.
결측치가 있는 컬럼은 : delta_15N입니다.
해당 컬럼에 총 3개의 결측치가 존재합니다.
결측치가 있는 컬럼은 : delta_13C입니다.
해당 컬럼에 총 3개의 결측치가 존재합니다.


In [136]:
#결측치가 있는 train data 행 확인
train_data[train_data.isna().sum(axis=1)>0]

,species,island,clutch_completion,culmen_length_mm,culmen_depth_mm,flipper_length_mm,sex,delta_15N,delta_13C,body_mass
6,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,42.0,20.2,190,NaN,9.13362,-25.09368,4250
8,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,34.1,18.1,193,NaN,NaN,NaN,3475
18,Adelie Penguin (Pygoscelis adeliae),Dream,No,39.8,19.1,184,MALE,NaN,NaN,4650
70,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,46.2,14.4,214,NaN,8.24253,-26.81540,4650
109,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,36.6,17.8,185,FEMALE,NaN,NaN,3700


In [137]:
# 결측치 처리 함수 범주형은 해당 행 삭제, 수치형은 평균 대체
def handle_na(data, missing_col):
    temp = data.copy()
    for col, dtype in missing_col:
        if dtype == 'O' : #관측치가 범주형일 경우 해당 행 삭제
            temp = temp.dropna(subset=[col])
        elif dtype == int or dtype == float:
            temp.loc[:,col] = temp[col].fillna(temp[col].mean())
    return temp

train_data = handle_na(train_data, missing_col)

#결측 처리 확인
missing_col = check_missing_col(train_data)

결측치가 존재하지 않습니다


In [153]:
#범주형 변수 코딩 변경
X_train = train_data[train_data.columns[0:9]]
y_train = train_data['body_mass']


X_train = pd.get_dummies(X_train)
X_train

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,delta_15N,delta_13C,species_Adelie Penguin (Pygoscelis adeliae),species_Chinstrap penguin (Pygoscelis antarctica),species_Gentoo penguin (Pygoscelis papua),island_Biscoe,island_Dream,island_Torgersen,clutch_completion_No,clutch_completion_Yes,sex_FEMALE,sex_MALE
0,50.0,15.3,220,8.305150,-25.190170,0,0,1,1,0,0,0,1,0,1
1,49.5,19.0,200,9.630740,-24.346840,0,1,0,0,1,0,1,0,0,1
2,45.1,14.4,210,8.519510,-27.018540,0,0,1,1,0,0,0,1,1,0
3,44.5,14.7,214,8.201060,-26.165240,0,0,1,1,0,0,0,1,1,0
4,49.6,16.0,225,8.383240,-26.842720,0,0,1,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,36.6,17.8,185,8.738543,-25.718804,1,0,0,0,0,1,0,1,1,0
110,39.2,18.6,190,9.110060,-25.795490,1,0,0,0,1,0,0,1,0,1
111,43.2,18.5,192,8.970250,-26.036790,1,0,0,0,1,0,0,1,0,1
112,46.9,16.6,192,9.805890,-24.737350,0,1,0,0,1,0,1,0,1,0


In [141]:
#테스트 데이터 결측 확인
print(check_missing_col(test_data))

결측치가 있는 컬럼은 : sex입니다.
해당 컬럼에 총 6개의 결측치가 존재합니다.
결측치가 있는 컬럼은 : delta_15N입니다.
해당 컬럼에 총 9개의 결측치가 존재합니다.
결측치가 있는 컬럼은 : delta_13C입니다.
해당 컬럼에 총 8개의 결측치가 존재합니다.
[['sex', dtype('O')], ['delta_15N', dtype('float64')], ['delta_13C', dtype('float64')]]


In [144]:
# 데스트 데이터 결측치 대체
missing_fill_val = {'sex' : 'MALE',
                   'delta_15N' : test_data.delta_15N.mean(),
                   'delta_13C' : test_data.delta_13C.mean()}

test_data = test_data.fillna(missing_fill_val)

check_missing_col(test_data)

결측치가 존재하지 않습니다


[]

In [156]:
#Test 데이터 변수 코딩 변경
X_test = pd.get_dummies(test_data)


In [157]:
# 데이터 정규화
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

In [158]:
#모델학습 - 로지스틱 회귀 분석
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_scaled_train,y_train)
pred_train = model.predict(X_scaled_train)
print("train data 모델 스코어 : " , model.score(X_scaled_train,y_train))

train data 모델 스코어 :  0.4594594594594595


In [159]:
#Best parameter 찾기
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'C':randint(low=0.001,high=100)}
random_search = RandomizedSearchCV(model,param_distributions=param_distribs,
                                  n_iter=100, cv=5)

random_search.fit(X_train,y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=100,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E71F785130>})

In [160]:
print('best parameter : {}'.format(random_search.best_params_))
print('best score : {:.4f}'.format(random_search.best_score_))

best parameter : {'C': 99}
best score : 0.1000


In [167]:
#모델학습 - 로지스틱 회귀 분석 (C=99) 하이퍼 파라미터 적용
from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression(C=99)
model2.fit(X_scaled_train,y_train)
pred_train2=model2.predict(X_scaled_train)
print("train data 모델 스코어 : " , model2.score(X_scaled_train,y_train))

train data 모델 스코어 :  0.9099099099099099


In [163]:
#RMSE
import numpy as np
from sklearn.metrics import mean_squared_error

MSE_train = mean_squared_error(y_train, pred_train2)
print("훈련 데이터 RSME : ", np.sqrt(MSE_train))

훈련 데이터 RSME :  155.20257033257482


In [172]:
# 훈련 모델 결과 적용
submission = pd.read_csv('./data/sample_submission.csv',encoding="UTF-8")
pred_test = model2.predict(X_scaled_test)
submission['Body Mass (g)'] = pred_test

submission.to_csv("./data/submission(final).csv", encoding="UTf-8",header=True, index= False)